In [6]:
import os
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('../Database/제품등록정보20211015.xlsx')
barcodes = df['바코드']
for barcode in tqdm(barcodes):
    os.makedirs(f'../Image/labels/{str(barcode)}', exist_ok=True)

100%|██████████| 23/23 [00:00<00:00, 1471.37it/s]


In [11]:
from flask import Flask
app = Flask('Salim OCR Result')

@app.route('/ocr')
def get_ocr_result(ocr_result):
    pass

if __name__ == '__main__':
    app.run()